In [2]:
import astroplan as ap
from astropy.coordinates import SkyCoord, Angle
from astropy.table import QTable
from astropy.time import Time
import astropy.units as u
from astroquery.gaia import Gaia

In [14]:
# # read in the text format version of the WDS and save as an Astropy QTable
# error_lines = []
# with open("wdsweb_summ2.txt", "r", newline="\n") as f:
#     for _ in range(5):
#         f.readline()
#     rows = []
#     for line in f.readlines():
#         try:
#             line = line.strip()  # remove EOL character
#             row = {}
#             row["ID"] = line[0:10]
#             row["Discoverer"] = line[10:17]
#             components_text = line[17:22]
#             if not components_text.strip() == "":
#                 row["Components"] = components_text
#             row["Num Obs"] = int(line[33:37])
#             row["Separation"] = (float(line[46:51]) + float(line[51:57])) / 2 * u.arcsec
#             row["Position Angle"] = int(line[42:45])
#             row["Mag A"] = float(line[58:63])
#             magb_text = line[63:69]
#             if not magb_text.strip() == ".":
#                 row["Mag B"] = float(magb_text)
#                 row["Mag Avg"] = (row["Mag A"] + row["Mag B"]) / 2
#             else:
#                 row["Mag Avg"] = row["Mag A"]
#             spectral_text = line[70:79]
#             if not spectral_text.strip() == "":
#                 row["Spectral Type"] = spectral_text
#             notes_text = line[107:111]
#             if not notes_text.strip() == "":
#                 row["Notes"] = notes_text
#             row["RA"] = Angle(f"{line[112:114]}h{line[114:116]}m{line[116:121]}s").to(u.deg)
#             row["Dec"] = Angle(f"{line[121:124]}d{line[124:126]}m{line[126:130]}s").to(u.deg)
#             pmra_text = line[80:84]
#             row["PM RA"] = u.Quantity(0 if pmra_text.strip() == "" else int(pmra_text), unit=u.mas / u.yr)
#             pmdec_text = line[84:88]
#             row["PM Dec"] = u.Quantity(0 if pmdec_text.strip() == "" else int(pmdec_text), unit=u.mas / u.yr)
#             rows.append(row)
#         except:
#             error_lines.append(line)
#             continue
# print(f"{len(error_lines)} errors and {len(rows)} lines successfully processed")
# print("Error lines")
# for line in error_lines:
#     print(line)

# wds = QTable(
#     rows,
#     names=[
#         "ID",
#         "Discoverer",
#         "Components",
#         "Num Obs",
#         "Separation",
#         "Position Angle",
#         "Mag A",
#         "Mag B",
#         "Mag Avg",
#         "Spectral Type",
#         "Notes",
#         "RA",
#         "Dec",
#         "PM RA",
#         "PM Dec",
#     ],
# )
# wds["Separation"].info.format = ".2f"
# wds["Mag A"].info.format = ".2f"
# wds["Mag B"].info.format = ".2f"
# wds["Mag Avg"].info.format = ".2f"
# wds["RA"].info.format = "12.8f"
# wds["Dec"].info.format = "13.8f"
# wds.write("wdsweb_summ2.ecsv", overwrite=True)
# wds

387 errors and 156474 lines successfully processed
Error lines
00056-1929TOI 378AB    2019 2019    1 312 312   3.3   3.3   .     .                                             000535.83-192920.2
00123-2729LDS2101      1954 2017    2  64  57   5.0   1.9 16.0  19.8            +085+027                    X         .         .
00172+0332LDS3014      1951 1951    1 294 294   1.5   1.5 18.3  20.2            -070-121                    X         .         .
00198-2334LDS2116      1954 1954    1  34  34  37.0  37.0 15.7  20.0            +056-047                    X         .         .
00199-2745GKM9007Aa,Ab 2022 2022    2  38  51   0.0   0.0   .     .                                           R 001952.22-274438.6
00293-5008TOI 337      2019 2019    1  61  61   0.1   0.1   .     .                                             002919.19-500824.4
00316+1344LDS9091      1961 1961    1 180 180   3.0   3.0 15.9  16.5                                       NX         .         .
00334-0033RST4740      1

ID,Discoverer,Components,Num Obs,Separation,Position Angle,Mag A,Mag B,Mag Avg,Spectral Type,Notes,RA,Dec,PM RA,PM Dec
,,,,arcsec,,,,,,,deg,deg,mas / yr,mas / yr
str10,str7,str5,int32,float64,int32,float64,float64,float64,str9,str4,float64,float64,float64,float64
00000+7530,A 1248,--,5,0.70,235,10.27,11.50,10.88,A7IV,--,0.02766667,75.48327778,34.0,5.0
00000+4004,ES 2543,AB,6,4.60,253,12.10,13.10,12.60,--,--,0.01525000,40.08872222,8.0,-4.0
00000+4004,ES 2543,AC,5,17.20,66,12.10,14.10,13.10,--,--,0.01525000,40.08872222,8.0,-4.0
00000+3852,BU 860,--,15,6.65,107,6.62,11.40,9.01,B9,--,0.00500000,38.85927778,3.0,-6.0
00000+0044,SKF1115,--,3,0.90,30,17.80,17.80,17.80,M1,--,359.99937500,0.72575000,-24.0,-20.0
00000-0530,OCC 622,--,0,0.10,-1,9.50,9.70,9.60,G5,--,0.00033333,-5.49436111,24.0,-12.0
00001+7727,LOC 122,--,6,8.15,249,14.10,16.60,15.35,--,T,0.03466667,77.44361111,86.0,37.0
00001+5400,ES 704,--,10,4.90,116,9.50,11.50,10.50,--,--,0.02850000,54.00005556,-30.0,3.0


In [13]:
wds = QTable.read("wdsweb_summ2.ecsv")
wds


<Angle [2.76666667e-02, 1.52500000e-02, 1.52500000e-02, ...,
        3.59978792e+02, 3.59963458e+02, 3.59973333e+02] deg>

# 2024-09-04 Updated procedure

As a result of the meeting, and emails with Jimmy, the following is the new procedure:

* Search the WDS for binaries, in RA 0h only, for the brightest match to the following criteria:
  * Separation under 0.5"
  * Magnitude brighter than 9
  * Magnitude difference less than 2
* Test if observable from APO during the A half of 2024-09-16.  If not, move on to the next brightest binary
* Search Gaia for a radius 110% of the separation of that binary, centered on the WDS coordinates
* If you don't get exactly 2 stars in that query, move on to the next brightest binary
* We now have Gaia's version of the binary.  Use the Gaia info for PA, separation, magnitudes, etc.
* From the midpoint of the Gaia binary, do a new Gaia search for point sources with the following criteria:
  * a 4' radius around the binary
  * within 1 mag of the brighter of the two components of the binary
* If there are no point sources found, move on to the next brightest binary
* For each of the possibly multiple point sources found, enter the binary & point source info in the targets table
* Move on to RA 1h and repeat the process
* Finish once the list of targets is a specified length, which might mean rotating through all the hours in RA multiple times

This avoids using coordinates from multiple catalogs simultaneously, and allows for the possibility of multiple point sources of varying separations for a given binary.

In [22]:
from astropy.coordinates import Angle
from astropy.table import QTable
import astropy.units as u

# WDS binary candidate criteria
dimmest_binary = 9
widest_binary = 0.5 * u.arcsec
binary_mag_difference = 1
binary_criteria = (
    (wds["Mag Avg"] < dimmest_binary)
    & (wds["Separation"] < widest_binary)
    & (abs(wds["Mag A"] - wds["Mag B"]) < binary_mag_difference)
)

# criteria to find matching gaia stars representing that binary
binary_match_cone = 30 * u.arcsec
binary_match_mag = .5

# point source criteria
source_match_mag = 2
source_match_sep = 8 * u.arcmin

# list size & observation details
num_targets = 20
observer = ap.Observer.at_site("Apache Point Observatory", timezone="US/Mountain")
observation_constraints = [
    ap.AltitudeConstraint(30 * u.deg, 80 * u.deg),
    ap.AirmassConstraint(2),
    # ap.AtNightConstraint.twilight_civil(),
]
session_beg, session_end = observer.tonight(Time("2024-09-16", format="iso", scale="utc"), horizon=-6 * u.deg)
session_end -= (session_end - session_beg) / 2 # only the A half of night

gaia_epoch_offset = 16 * u.yr

wds["Used"] = False
targets = []
hour_angle = u.Quantity(0, unit=u.deg)
while len(targets) < num_targets:
    binaries = wds[
        binary_criteria
        & (wds["RA"] >= hour_angle)
        & (wds["RA"] < hour_angle + 15 * u.deg)
        & (wds["Separation"] > 0)
        & (wds["Used"] == False)]
    binaries.sort("Mag Avg")
    print(f"RA={Angle(hour_angle).hour:.0f}h has {len(binaries)} binaries")
    for binary in binaries:
        print(f"Looking at {binary["ID"]}", end=" ")
        binary["Used"] = True
        wds_coord = SkyCoord(binary["RA"], binary["Dec"], pm_ra_cosdec=binary["PM RA"], pm_dec=binary["PM Dec"], unit="deg")
        observable = ap.is_observable(observation_constraints, observer, wds_coord, (session_beg, session_end))
        if not observable:
            print("Not observable")
            continue
        wds_gaia_epoch = wds_coord.apply_space_motion(dt=gaia_epoch_offset)
        binary_query = (
            f"select *"#DESIGNATION, ra, dec, parallax, astrometric_n_good_obs_al, ruwe, phot_g_mean_mag "
            f"  from gaiadr3.gaia_source "
            f"  WHERE CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {wds_gaia_epoch.ra.value}, {wds_gaia_epoch.dec.value}, {binary_match_cone.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag < {binary["Mag A"] + binary_match_mag} "
            f"    and phot_g_mean_mag > {binary["Mag A"] - binary_match_mag} "
        )
        binary_matches = Gaia.launch_job(binary_query).get_results()
        match len(binary_matches):
            case 0:
                print("Not found")
                continue
            case 1:
                pass
            case _:
                print(f"Too many Gaia stars ({len(binary_matches)})")
                print(binary_matches)
                continue
        gaia_match = binary_matches[0]
        gaia_mag = gaia_match["phot_g_mean_mag"]
        if abs(gaia_mag - binary["Mag A"]) > binary_match_mag:
            print("Mag too different")
            continue
        gaia_coord = SkyCoord(
            gaia_match["ra"],
            gaia_match["dec"],
            pm_ra_cosdec=gaia_match["pmra"] * u.mas / u.yr,
            pm_dec=gaia_match["pmdec"] * u.mas / u.yr,
            unit="deg")#.apply_space_motion(dt=-gaia_epoch_offset)

        # we now have the gaia version of the binary
        # look for nearby point sources of similar magnitude
        source_query = (
            f"select * "
            f"  from gaiadr3.gaia_source "
            f"  where CONTAINS( "
            f"      POINT('ICRS', ra, dec), "
            f"      CIRCLE('ICRS', {gaia_coord.ra.value}, {gaia_coord.dec.value}, {source_match_sep.to(u.deg).value})) = 1 "
            f"    and phot_g_mean_mag > {gaia_mag - source_match_mag} "
            f"    and phot_g_mean_mag < {gaia_mag + source_match_mag} "
        )
        source_matches = Gaia.launch_job(source_query).get_results()
        if len(source_matches) == 0:
            print("No point sources")
            continue
        gaia_coord = gaia_coord.apply_space_motion(dt=-gaia_epoch_offset)
        target_found = False
        for source in source_matches:
            source_coord = SkyCoord(
                source["ra"],
                source["dec"],
                pm_ra_cosdec=source["pmra"] * u.mas / u.yr,
                pm_dec=source["pmdec"] * u.mas / u.yr,
                unit="deg").apply_space_motion(dt=-gaia_epoch_offset)
            if source["DESIGNATION"] == "Gaia DR3 441161693272718336":
                pass
            if source_coord.separation(gaia_coord) < 1 * u.mas:
                print(f"Skipping gaia match of binary (of {len(source_matches)} total sources)")
                continue
            target = {}
            target["Binary WDS ID"] = binary["ID"]
            target["Binary RA"] = gaia_coord.ra.to_string(unit=u.hour, precision=2)
            target["Binary Dec"] = gaia_coord.dec.to_string(unit=u.deg, precision=2)
            target["Binary Sep"] = binary["Separation"].to(u.arcsec)
            target["Binary PA"] = 0 # TODO: fixme
            target["Binary Mag A"] = binary["Mag A"]
            target["Binary Mag B"] = binary["Mag B"]
            target["Point ID"] = source["DESIGNATION"]
            point_coord = SkyCoord(
                source["ra"],
                source["dec"],
                pm_ra_cosdec=source["pmra"] * u.mas / u.yr,
                pm_dec=source["pmdec"] * u.mas / u.yr,
                unit="deg").apply_space_motion(dt=-gaia_epoch_offset)
            target["Point RA"] = point_coord.ra.to_string(unit=u.hour, precision=2)
            target["Point Dec"] = point_coord.dec.to_string(unit=u.deg, precision=2)
            point_sep = gaia_coord.separation(point_coord)
            target["Point Sep"] = f"{point_sep.dms[1]:.0f}m{point_sep.dms[2]:02.0f}s"
            target["Point PA"] = gaia_coord.position_angle(point_coord).to(u.deg)
            target["Point Mag"] = source["phot_g_mean_mag"]
            targets.append(target)
            print(f"Match found: {source["DESIGNATION"]}, target count={len(targets)}")
            target_found = True
        if target_found:
            break
    hour_angle = u.Quantity((hour_angle.value + 15) % 360, unit=u.deg)
wds.remove_column("Used")

diffit_targets = QTable(targets)
diffit_targets["Binary Sep"].info.format = ".2f"
diffit_targets["Binary PA"].info.format = ".2f"
diffit_targets["Point PA"].info.format = ".0f"
diffit_targets["Point Mag"].info.format = ".2f"
diffit_targets

RA=0h has 42 binaries
Looking at 00318+5431 

c:\Users\User\anaconda3\envs\Astro\Lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "pmsafe" yielded 1 of "distance overridden (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


Not found
Looking at 00315-6257 Not observable
Looking at 00568+6022 Skipping gaia match of binary (of 1 total sources)
Looking at 00243+5201 Skipping gaia match of binary (of 1 total sources)
Looking at 00373-2446 Too many Gaia stars (2)
    solution_id             DESIGNATION          ... libname_gspphot
                                                 ...                
------------------- ---------------------------- ... ---------------
1636148068921376768 Gaia DR3 2347260998052761088 ...                
1636148068921376768 Gaia DR3 2347272641708189440 ...                
Looking at 00003-4417 Not observable
Looking at 00284-2020 Not found
Looking at 00153+4412 Not found
Looking at 00327+3851 Not found
Looking at 00290+4825 Not found
Looking at 00452+0015 Not found
Looking at 00374-3717 Not observable
Looking at 00358+4901 Not found
Looking at 00345-0433 Not found
Looking at 00541+6626 Skipping gaia match of binary (of 1 total sources)
Looking at 00257+0741 Not found
Looking at 00

Binary WDS ID,Binary RA,Binary Dec,Binary Sep,Binary PA,Binary Mag A,Binary Mag B,Point ID,Point RA,Point Dec,Point Sep,Point PA,Point Mag
,,,arcsec,,,,,,,,deg,
str10,str12,str13,float64,int32,float64,float64,str28,str12,str13,str5,float64,float32
00541+5701,0h54m05.10s,57d00m50.67s,0.20,0.00,7.9,8.79,Gaia DR3 424197122217157248,0h54m59.09s,56d59m35.29s,7m27s,100,9.76
01137+0735,1h13m43.89s,7d34m31.30s,0.20,0.00,5.6,6.4,Gaia DR3 2577061260424130304,1h13m45.25s,7d34m41.68s,0m23s,63,6.18
02186+4017,2h18m35.36s,40d16m48.00s,0.15,0.00,7.9,7.9,Gaia DR3 332903778403980544,2h18m36.25s,40d16m43.60s,0m11s,113,7.15
02186+4017,2h18m35.36s,40d16m48.00s,0.15,0.00,7.9,7.9,Gaia DR3 332910611694486272,2h18m50.01s,40d23m46.79s,7m31s,22,9.74
03470+2431,3h46m59.39s,24d31m12.45s,0.15,0.00,6.81,6.9,Gaia DR3 66791246876452736,3h47m24.08s,24d35m18.51s,6m57s,54,7.63
04433+5931,4h43m17.78s,59d31m15.41s,0.45,0.00,7.25,7.42,Gaia DR3 279156420223004416,4h42m43.85s,59d29m18.42s,4m44s,246,8.53
05047+7404,5h04m39.68s,74d04m01.14s,0.30,0.00,6.53,7.29,Gaia DR3 502547290015662848,5h06m31.72s,74d05m31.46s,7m50s,79,7.70
13329+3454,13h32m50.93s,34d54m25.48s,0.30,0.00,7.27,8.08,Gaia DR3 1470143738937281536,13h32m46.27s,34d56m13.71s,2m02s,332,9.04
